### I. Let's create our first Dataframe

- Think of `DataFrame` as tables __\*but\*__ with a rich set of functionalities for data analysis and manipulation
- It is the main data structure of the `pandas` library
- Has tabular properties such as rows and columns including indices

In [ ]:
# Bring in the library
import pandas as pd

In [ ]:
# Let's instantiate a dataframe with dummy data
df = pd.DataFrame([[1,2,3],[4,5,6],[7,8,9]], columns=["A","B","C"])

In [ ]:
# It understands dictionaries too, altenatively...
data = {
    'A': [1, 4, 7],
    'B': [2, 5, 8],
    'C': [3, 6, 9]
}

df = pd.DataFrame(data)

In [ ]:
# Ending a code cell with a `DataFrame` will preview of its contents
df

#### 0, 1, 2...?

- These are called *__indices__*
- They are used to __uniquely identify a row__ and pandas automatically assigned them for us by default
- Think of them as row labels (and yep, they are customizable)

In [ ]:
# Customize indices to your liking
pd.DataFrame([[1,2,3],[4,5,6],[7,8,9]], columns=["A","B","C"], index=["x","y","z"])

### II. Describing Dataframes

Let's look at common functions to examine our dataframes

In [ ]:
# To show the dimensions of the dataframe
df.shape

In [ ]:
# To show the # of elements
df.size

In [ ]:
# To see the first or last N rows
df.head() # or df.tail()

In [ ]:
# To sample the dataset for previewing
df.sample(3)

In [ ]:
# To show a summary of the composition of the dataframe
df.info()

In [ ]:
# To show some basic statistics
df.describe()

In [ ]:
# To show the unique # of elements
df.nunique()

# To apply for a specific column
# df['A'].unique()

In [ ]:
# To get the indices in an array
df.index.tolist()

### III. From Files to DataFrames

- A more common use case in the real world however would be to load data from files
- Pandas supports tons of file formats which can be loaded easily through the `.read_*` function
- Using the same exact __Olympics Results__ dataset in different file formats, let's see it in action!

In [ ]:
# Read CSV file
results = pd.read_csv('./data/olympics_results.csv')
results.head()

In [ ]:
# Read Excel file
results = pd.read_excel('./data/olympics_data.xlsx')
results.head()

In [ ]:
# It loaded the default sheet, but we can specify a specific sheet
results = pd.read_excel('./data/olympics_data.xlsx', sheet_name="results")
results.head()

In [ ]:
# Read feather file
result = pd.read_feather('./data/olympics_results.feather')
result.head()

In [ ]:
# Read parquet file
results = pd.read_parquet('./data/olympics_results.parquet')
results.head()

#### Knowledge check 💡

_Notice any differences when dealing with various file formats using the same exact dataset?_

### IV. From DataFrames to Files

- We can also export data back to files with pandas
- Useful for saving processed/cleaned data
- This is done through the `.to_*` function

In [ ]:
# Export dataframe into JSON file
results.to_json('./results.json')

### V. Accessing Data

- Selecting elements from a dataframes can commonly be done using Python's slice notation ("`:`")
- Use `.iloc` for integer-based indexing and `.loc` for label-based indexing
- Use `.iat` or `.at` for fast scalar value access (single cell)
- Use these native indexer methods to efficiently select elements in dataframes whenever you can!
- For these purposes, we'll be using our __toy dataset__

In [ ]:
coffee = pd.read_csv('./coffee.csv')
coffee

In [ ]:
# To select specific rows using index
coffee.iloc[[0,1,5]]

In [ ]:
# To select rows from index 10 onwards
coffee.iloc[10:]

In [ ]:
# To select rows using start and stop positions
coffee.iloc[10:12]

#### Knowledge check 💡

_Pandas is actually following Python's zero-based indexing when slicing and it uses an exclusive upper bound. Why do you think this is done this way?_


In [ ]:
# To select all rows
coffee.iloc[:]

In [ ]:
# To select all rows with specific columns
coffee.iloc[:, ['Day', 'Coffee Type']]

In [ ]:
# .loc to the rescue
coffee.loc[:, ['Day', 'Coffee Type']]

In [ ]:
# Using index labels instead of default integers
coffee.index = coffee["Day"]
coffee.head()

In [ ]:
# .loc works with labels
coffee.loc["Monday":"Wednesday", "Units Sold"]

In [ ]:
coffee = pd.read_csv('./coffee.csv') # reset df

# Use .iat or .at for selecting specific cells
coffee.iat[0,0]
coffee.at[0,"Day"]

In [ ]:
# To select all rows using a specific column
coffee.Day

In [ ]:
# But square brackets are more robust because it supports both single and multiple worded columns
coffee['Units Sold']

In [ ]:
coffee.sort_values(["Units Sold", "Coffee Type"], ascending=[0,1])

In [ ]:
# You can manually iterate on the rows but USE SPARINGLY
# Not using the builtin methods loses the memory and performance benefits of pandas
for index, row in coffee.iterrows():
    print(index)
    print(row)
    # print(row["Coffee Type"])
    print("---")

### VI. Filtering Data

- Accessing data requires precise selection whereas filtering allows us to describe certain __criteria__ to match elements
- This is mostly done using __boolean indexing__ and __comparison__ operators
- For this chapter, we'll use the __Olympics Biography__ dataset to explore filtering techniques

In [ ]:
# Here's our dataset
bios = pd.read_csv('./data/olympics_bios.csv')
bios.info()

In [ ]:
# To select rows from specific columns based on height condition
df = bios.loc[bios['height_cm'] > 215, ['name', 'height_cm']]
df.head()

In [ ]:
# Short-hand notation
df = bios[bios['height_cm'] > 215][['name', 'height_cm']]
df.head()

In [ ]:
# To filter using multiple conditions
bios[(bios['height_cm'] > 215)  & (bios['born_country'] == 'USA')]

In [ ]:
# To filter using string operations
bios[bios['name'].str.contains("maron", case=False)]

In [ ]:
# To filter names with repeated letters using regex
repeated_letters = bios[bios['name'].str.contains(r'(.)\1', na=False)]
repeated_letters.head()

In [ ]:
# To filter names with 'son' or 'sen' at the end using regex
son_sen_names = bios[bios['name'].str.contains(r'son$|sen$', case=False, na=False)]
son_sen_names.head()

In [ ]:
# To filter athletes from the 90's using regex
born_90s = bios[bios['born_date'].str.contains(r'^199', na=False)] # ignore NaN values
born_90s.head()

In [ ]:
# You can mix and match things!
bios[bios['born_country'].isin(["PHI"]) & (bios['name'].str.startswith("Hidilyn"))]

### VII. Manipulating DataFrames

- Adding, modifying or dropping columns may become necessary as we make our dataset cleaner and more robust
- We can store derived values from existing columns into a new column (e.g market_cap = price * shares)
- Irrelevant or redundant columns can be dropped
- Let's see some examples in action using our toy dataset

In [ ]:
# To add a new column with fixed values
coffee['price'] = 100
coffee.head()

In [ ]:
# To add the price with a smarter approach using numpy's conditional where
import numpy as np

coffee['new_price'] = np.where(coffee['Coffee Type']=='Espresso', 100, 150)
coffee.head()

In [ ]:
# Time to delete the previous column
coffee.drop(columns=['price'])

In [ ]:
# Wait what-- the old price column is still here
coffee.head()

In [ ]:
# Dataframes are immutable by default, so drop() returned a new copy. We can set inplace to True to override this.
coffee.drop(columns=['price'], inplace=True)

# This is perfectly fine as well
# coffee = coffee.drop(columns=['price'])

# Alternatively...
# coffee = coffee[['Day', 'Coffee Type', 'Units Sold', 'new_price']]

coffee

In [ ]:
# To add a new column and deriving its values based on existing columns
coffee ['revenue'] = coffee['Units Sold'] * coffee['new_price']
coffee

In [ ]:
# To rename a column
coffee.rename(columns={'new_price': 'price'}, inplace=True)
coffee.head()

In [ ]:
# Another example: Let's store the first name as a new column
bios['first_name'] = bios['name'].str.split(' ').str[0]
bios.head()

In [ ]:
bios.query('first_name == "Juan"')

In [ ]:
# To add a new column and store values based on custom logic using python lambdas
bios['height_category'] = bios['height_cm'].apply(lambda x: 'Short' if x < 165 else ('Average' if x <185 else 'Tall'))
bios.head()

In [ ]:
# To define and use regular functions instead of lambdas
def categorize_athlete(row):
    if row['height_cm'] < 175 and row['weight_kg'] < 70:
        return 'Lightweight'
    elif row ['height_cm'] < 185 or row['weight_kg'] <= 80:
        return 'Middleweight'
    
    else:
        return 'Heavyweight'

bios['weight_category'] = bios.apply(categorize_athlete, axis=1) # 1 is rows, 0 is columns
bios.head()

### VIII. Combining DataFrames

#### Merging

- In the real world, typical datasets are normalized and split into compact, purposeful tables
  - For example, sales records for a coffee shop can be stored in one dataframe, while the menu can be in another
- Depending on our use case, we'd want to combine various dataframes to derive meaningful insights
- It is important to identify a column from a dataframe that **relates** to a column in another dataframe
- But how do we exactly merge two datasets in pandas? This is done through `pd.merge()`.
- Think of `pd.merge()` like combining two spreadsheets based on a common column, similar to matching information from two lists.

##### Type of Joins in Pandas

<img src="images/pandas_basic_joins.png" alt="BMI Formula" width="500" />

- **INNER JOIN:** Only keeps rows where the matching column exists in BOTH tables
  - Like finding common friends between two people
- **LEFT JOIN:** Keeps ALL rows from the left table, even if no match in right table
  - Like keeping your full class list and adding grades where available
- **RIGHT JOIN:** Keeps ALL rows from the right table, even if no match in left table-
  - Like keeping all grades, even for transferred students not in current roster
- **OUTER JOIN:** Keeps ALL rows from BOTH tables
  - Like combining two class rosters completely

In [ ]:
# Observe how "NOC" column relates to the "born_country" column in our Olympics Biography dataset
nocs = pd.read_csv('./data/noc_regions.csv')
nocs.head()

In [ ]:
# First, we specify the two dataframes
# Then, we specify their respective columns that need to match values across rows
# Finally, we specify the join type to apply for the merge
# LEFT JOIN will keep all rows from "bios" even if there is no match in "nocs" (they will simply be NAs)
bios_new = pd.merge(bios, nocs, left_on='born_country', right_on='NOC', how='left')

In [ ]:
# Notice that pandas automatically resolves conflicting column names by adding "_<suffix>"
# Making the column name more meaningful in the context of Olympics Biography
bios_new.rename(columns={'region': 'born_country_full'}, inplace=True)
bios_new.head()

In [ ]:
# With this new information, we can gather more insights such as
# Finding atheletes who competed under a different region
bios_new[bios_new['NOC_x'] != bios_new['born_country_full']][['name','NOC_x','born_country_full']]

#### Concatenation

- Perfect for combining similar data structures
- Like stacking blocks on top of each other (rows) or side by side (columns)
- This is done through `pd.concat()`.

In [ ]:
# To concatenate two filtered sub-dataframes from the same origin dataframe
us_df = bios[bios['born_country'] == 'USA'].copy()
ph_df = bios[bios['born_country'] == 'PHI'].copy()

combined_df = pd.concat([us_df,ph_df])

In [ ]:
combined_df

#### Knowledge check 💡

_What do you think is the main difference between merging and concatenating dataframes?_

### IX. Aggregating Data

- Aggregating data enables us to extract insights and identify patterns crucial for data analysis
- Use cases include:
    - Grouping data by categories
    - Identifying trends and distributions
    - Performing statistical analysis (mean, median, standard deviation, etc)
    - Providing business metrics (total sales, average revenue, etc)
- In this chapter, let's revisit our __Olympics Biography__ and Toy datasets

In [ ]:
bios.head()

In [ ]:
# To count unique values of a column
bios['born_city'].value_counts() # budapest leading the charge

In [ ]:
# Determine state that has the most # of athletes
bios[bios['born_country'] == 'USA']['born_region'].value_counts()

In [ ]:
# To get the total unit sold grouped by coffee type
coffee.groupby(['Coffee Type'])['Units Sold'].sum()

In [ ]:
# To get multiple aggregations with the same group by
coffee.groupby(['Coffee Type']).agg({'Units Sold': 'sum', 'price': 'mean'})

In [ ]:
# To get multiple aggregations with multiple group by
coffee.groupby(['Coffee Type', 'Day']).agg({'Units Sold': 'sum', 'price': 'mean'})

In [ ]:
# Let's try grouping by born_date
bios.head()

In [ ]:
# To group by born_date and get unique name count
bios['born_date'] = pd.to_datetime(bios['born_date']) # onvert from string to proper pandas datetime format
bios.groupby(bios['born_date'].dt.year)['name'].count()

In [ ]:
# To reset the index with defaults
bios.groupby(bios['born_date'].dt.year)['name'].count().reset_index()

In [ ]:
# To sort such that the highest count is at the top
bios.groupby(bios['born_date'].dt.year)['name'].count().reset_index().sort_values('name', ascending=False)

### X. Other Useful Operations

There is a myriad of functionalities offered in pandas which you can find more about in their [official documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html#user-guide). However, here are some more useful functionalities that might come handy at your disposal. 🙂

#### Pivoting

- Powerful way to reshape your data by turning unique column values as new columns
- Helps transform data from "long" to "wide" format
- This can be achieved through `df.pivot`, but requires unique index/value combinations
- Consider `pd.pivot_table` to reshape and aggregate duplicate values in one go

In [ ]:
# Back to our toy dataset
coffee

In [ ]:
# Let's make the Coffee Type as columns and revenue as the rows
pivoted = coffee.pivot(columns='Coffee Type', index='Day', values='revenue')

In [ ]:
pivoted # ...how is this useful? Well, it allows for different perspective on data

In [ ]:
# How much sales we got from Latte on Monday?
pivoted.loc['Monday', 'Latte']

In [ ]:
# What's our total sales per product?
pivoted.sum()

In [ ]:
# What's our total sales per business day?
pivoted.sum(axis=1)

#### Pandas DateTime

- It's a good practice to represent date and datetime values in actual datetime objects
- This allows for:
  - More efficient operations
  - Easy date arithmetic
  - Consistent format handling
  - Timezone handling
  - Integration with timeseries tools from pandas and other libraries
- Looking back at our __Olympics Biography__ dataset, we see that `born_date` is of type `object` (string)...

In [ ]:
bios.info()

In [ ]:
# Let's try to add a proper datetime column
bios['born_datetime'] = pd.to_datetime(bios['born_date'])
bios.head()

In [ ]:
# While values appear similar, its type is now datetime64
bios.info()

In [ ]:
# Pandas datetime unlocks a rich set of datetime utilities

# Store the year
bios['born_year'] = bios['born_datetime'].dt.year

# Derive the athlete's age, null out if deceased
bios['age'] = np.where(bios['died_date'].isna(), pd.Timestamp.now().year - bios['born_year'], np.nan)
bios[['name','born_year', 'age']]

#### Handling Null values

It is common for certain datasets to have null values in some columns. Depending on the use case, you may choose to (1) ignore them, (2) fill them with a default value, or (3) drop these rows with null values.

Let's examine our __Olympics Biography__ dataset...

In [ ]:
# Notice that non-null values are not always similar in count? That is because there's likely null values in some columns.
bios.info()

In [ ]:
# Surely enough...
bios.isna().sum()

In [ ]:
# NaN = Not a Number
bios.sample(10)

In [ ]:
# To drop rows with missing height and/or weight information
bios.dropna(subset=['height_cm', 'weight_kg'])

In [ ]:
# To fill missing height information with a default value, i.e median
bios.fillna(bios['height_cm'].median())

#### Ranking Data

Dataframes also allow us to easily compute for numerical ranks:

In [ ]:
# Ranking them by height and see what we got?
bios['height_rank'] = bios['height_cm'].rank(ascending=False)
bios.sort_values('height_rank')

### Epilogue: Brainteaser 🧠

How about quick warm up exercises to enforce some concepts before we jump into the group work?

#### Find Hidilyn Diaz's Olympic Records

In [ ]:
# Start wrangling here

#### Find the GOAT

In [ ]:
# Start wrangling here